In [ ]:
from pypdf import PdfReader
import os
from tqdm import tqdm

#print out books in pdf folder
print("Books in pdf folder:")
for file in os.listdir("pdfs"):
    print(file)



file_name = input("Enter the name of the file you want to extract text from: ")
book_name = input("Enter the name of the book to make into file directory: ")


#make file directory if it doesn't exist
if not os.path.exists(book_name):
    os.makedirs(book_name)

#make page_chunks directory if it doesn't exist
if not os.path.exists(f"{book_name}/page_chunks"):
    os.makedirs(f"{book_name}/page_chunks")



reader = PdfReader("pdfs/" + file_name)
text_chunks = []

first_run = True

for page in reader.pages:
    

    if first_run:
        first_run = False

        first_page = page.extract_text() + "\n"

        second_page = page.extract_text() + "\n"

        text_chunks.append(first_page + second_page)

    else:
        first_page = second_page
        second_page = page.extract_text() + "\n"

        text_chunks.append(first_page + second_page)


for index, file_chunk in enumerate(text_chunks):
    with open(f"{book_name}/page_chunks/text_chunks_{index}.txt", "w", encoding="utf-8") as f:
        f.write(file_chunk)

In [ ]:
import os
import openai
import dotenv


dotenv.load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")


#print("Books in pdf folder:")
#for file in os.listdir("pdfs"):
#    print(file)
#
#print("-----------------------------------")

print("Folders in current directory")
for file in os.listdir():
    if os.path.isdir(file) and file != "pdfs":
        print(file)



folder_name = input("Enter the name of the folder you want to extract text from: ")



#get the list of all files in the directory page_chunks
files = os.listdir(f"{folder_name}/page_chunks")




start_sequence = "\nAI:"
restart_sequence = "\nHuman: "

if not os.path.exists(f"{folder_name}/flash_chunks"):
    os.makedirs(f"{folder_name}/flash_chunks")

if not os.path.exists(f"{folder_name}/file_exceptions"):
    os.makedirs(f"{folder_name}/file_exceptions")  


for index, file in tqdm(enumerate(files[2:])):
    
    try:
        file_content = open(f"{folder_name}/page_chunks/{file}", "r", encoding="utf-8").read()


        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages= [{"role": "system", "content": "You produce flashcards from a two-page section from a book. You produce highly detailed flash cards with a term name and a definition in the format: Term: <Card Name> \n Definition: <Card Definition> \n ... Term: <Card Name> \n Defnition: <Card Definition>"}, {"role": "user", "content": "please produce some flashcards from the provided content: \n" + file_content}],
        )

        with open(f"{folder_name}/flash_chunks/flash_chunks_{index}.txt", "w", encoding="utf-8") as f:
            f.write(response["choices"][0]["message"]["content"])

    except Exception as e:

        # write file exepction to a file in file_exceptions folder
        with open(f"{folder_name}/file_exceptions/{file}.txt", "w", encoding="utf-8") as f:
            f.write(str(e))
        



    






In [ ]:
import os
import pandas as pd


card_deck = input("Enter the name of the card deck: ")
card_deck = "like_switch"

card_files = os.listdir(f"C:/Users/malco/OneDrive/Documents/GitHub/Auto-GPT/GPT-Tools/book_extraction/flash_chunks/{card_deck}")




# make a csv file in flash_deck folder
df = pd.DataFrame(columns=["card_name", "card_definition"])

    






for card_file in tqdm(card_files):
    with open(f"C:/Users/malco/OneDrive/Documents/GitHub/Auto-GPT/GPT-Tools/book_extraction/flash_chunks/{card_deck}/{card_file}", "r", encoding="utf-8") as f:
        
        try:
            cards = f.read().split("\n\n")
            
            for card in cards:
                card = card.strip()
                card_name, card_definition = card.split("\n")


                # if card name matches the regex of Term: <Card Name> and if card definition matches the regex of Definition: <Card Definition>
                if card_name.startswith("Term: ") and card_definition.startswith("Definition: "):

                    card_name = card_name.replace("Term: ", "")
                    card_definition = card_definition.replace("Definition: ", "")


                    card_name = card_name.replace('"', "")
                    card_name = card_name.replace("”", "")
                    card_name = card_name.replace("“", "")
                    card_name = card_name.strip()
                    card_name = card_name.lower()

                    #card_definition = card_definition.replace('"', "'")
                    #card_definition.replace("”", " ")
                    #card_definition.replace("“", " ")

                    #card_definition = '"' + card_definition + '"'

                    

                    # add data to csv file
                    card_row = pd.DataFrame({"card_name": [card_name], "card_definition": [card_definition]})
                    df = pd.concat([df, card_row], ignore_index=True)
            
        except Exception as e:
            continue


df = df.sort_values(by=["card_name"])


df.to_csv(f"C:/Users/malco/OneDrive/Documents/GitHub/Auto-GPT/GPT-Tools/book_extraction/flash_decks/{card_deck}.csv", index=False)

In [ ]:
import pandas
import os

# list folders in directory C:/Users/malco/OneDrive/Documents/GitHub/Auto-GPT/GPT-Tools/book_extraction/

folders = os.listdir("C:/Users/malco/OneDrive/Documents/GitHub/Auto-GPT/GPT-Tools/book_extraction/")

#print out folder names

print("Folder Names:")
for folder in folders:
    print(folder)


df = pandas.read_csv(f"C:/Users/malco/OneDrive/Documents/GitHub/Auto-GPT/GPT-Tools/book_extraction/{name}/flash_decks/{name}.csv")

# group by card name and aggregate card definitions
df = df.groupby("card_name").agg({"card_definition": lambda x: "\n\n".join(x)})

df.to_csv(f"C:/Users/malco/OneDrive/Documents/GitHub/Auto-GPT/GPT-Tools/book_extraction/{name}/flash_decks/{name}_aggr.csv")

In [ ]:
import pandas as pd
import openai
import dotenv
import os

dotenv.load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")


df = pd.read_csv("C:/Users/malco/OneDrive/Documents/GitHub/Auto-GPT/GPT-Tools/book_extraction/flash_decks/like_switch_aggr.csv")

#for value in data frame column card_definition has \n filter into a new data frame


df = df[df["card_definition"].str.contains("\n")]


reduced_df = pd.DataFrame(columns=["card_name", "card_definition"])

for index, row in df.iterrows():
    
    #try:
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages= [{"role": "system", "content": 
                    "You take in information and condense it into one high quality flashcard. You produce highly a detailed flashcard with a term name and a definition in the format: Term: <Card Name> \n\n Definition: <Card Definition> "}, 
                    {"role": "user", "content": f"Please produce a flashcard on {row['card_name']} from the following content: \n {row['card_definition']}"}],
    )

    
    
    text = response["choices"][0]["message"]["content"]

    print(text)
    term, definition = text.split("\n\n")

    term = term.replace("Term: ", "")
    definition = definition.replace("Definition: ", "")

    reduced_df = pd.concat([reduced_df, pd.DataFrame({"card_name": [term], "card_definition": [definition]})], ignore_index=True)

    reduced_df["card_definition"] = reduced_df["card_definition"].apply(lambda x: x.strip())

# save reduced_df to a csv file
reduced_df.to_csv("C:/Users/malco/OneDrive/Documents/GitHub/Auto-GPT/GPT-Tools/book_extraction/flash_decks/like_switch_reduced.csv")
